# 实例：cats_vs_dogs 图像分类 
以下代码以猫狗图片二分类任务为示例，展示了使用 tf.data 结合 tf.io 和 tf.image 建立 tf.data.Dataset 数据集，并进行训练和测试的完整过程。数据集可至 这里 下载。使用前须将数据集解压到代码中 data_dir 所设置的目录（此处默认设置为 C:/datasets/cats_vs_dogs ，可根据自己的需求进行修改）。

In [1]:
import tensorflow as tf
import os

num_epochs = 10
batch_size = 32
learning_rate = 0.001
data_dir = '/DATA/data_Cats-vs-Dogs'
train_cats_dir = data_dir + '/train/cats/'
train_dogs_dir = data_dir + '/train/dogs/'
test_cats_dir = data_dir + '/valid/cats/'
test_dogs_dir = data_dir + '/valid/dogs/'

def _decode_and_resize(filename, label):
    image_string = tf.io.read_file(filename)            # 读取原始文件
    image_decoded = tf.image.decode_jpeg(image_string)  # 解码JPEG图片
    image_resized = tf.image.resize(image_decoded, [256, 256]) / 255.0
    return image_resized, label

def train():
    train_cat_filenames = tf.constant([train_cats_dir + filename for filename in os.listdir(train_cats_dir)])
    train_dog_filenames = tf.constant([train_dogs_dir + filename for filename in os.listdir(train_dogs_dir)])
    train_filenames = tf.concat([train_cat_filenames, train_dog_filenames], axis=-1)
    print(train_filenames)
    train_labels = tf.concat([
        tf.zeros(train_cat_filenames.shape, dtype=tf.int32), 
        tf.ones(train_dog_filenames.shape, dtype=tf.int32)], 
        axis=-1)

    train_dataset = tf.data.Dataset.from_tensor_slices((train_filenames, train_labels))
    train_dataset = train_dataset.map(
        map_func=_decode_and_resize, 
        num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # 取出前buffer_size个数据放入buffer，并从其中随机采样，采样后的数据用后续数据替换
    train_dataset = train_dataset.shuffle(buffer_size=23000)    
    train_dataset = train_dataset.batch(batch_size)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(256, 256, 3)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(32, 5, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        metrics=[tf.keras.metrics.sparse_categorical_accuracy]
    )

    model.fit(train_dataset, epochs=num_epochs)
    
    # 构建测试数据集
    test_cat_filenames = tf.constant([test_cats_dir + filename for filename in os.listdir(test_cats_dir)])
    test_dog_filenames = tf.constant([test_dogs_dir + filename for filename in os.listdir(test_dogs_dir)])
    test_filenames = tf.concat([test_cat_filenames, test_dog_filenames], axis=-1)
    test_labels = tf.concat([
        tf.zeros(test_cat_filenames.shape, dtype=tf.int32), 
        tf.ones(test_dog_filenames.shape, dtype=tf.int32)], 
        axis=-1)

    test_dataset = tf.data.Dataset.from_tensor_slices((test_filenames, test_labels))
    test_dataset = test_dataset.map(_decode_and_resize)
    test_dataset = test_dataset.batch(batch_size)

    print(model.metrics_names)
    print(model.evaluate(test_dataset))
    
if __name__ == '__main__':
    # 构建训练数据集
    train()

tf.Tensor(
[b'/DATA/data_Cats-vs-Dogs/train/cats/cat.5324.jpg'
 b'/DATA/data_Cats-vs-Dogs/train/cats/cat.5814.jpg'
 b'/DATA/data_Cats-vs-Dogs/train/cats/cat.230.jpg' ...
 b'/DATA/data_Cats-vs-Dogs/train/dogs/dog.6260.jpg'
 b'/DATA/data_Cats-vs-Dogs/train/dogs/dog.6535.jpg'
 b'/DATA/data_Cats-vs-Dogs/train/dogs/dog.201.jpg'], shape=(23000,), dtype=string)
Train for 719 steps
Epoch 1/10
719/719 [==============================] - 78s 108ms/step - loss: 0.7330 - sparse_categorical_accuracy: 0.5511
Epoch 2/10
719/719 [==============================] - 77s 107ms/step - loss: 0.6408 - sparse_categorical_accuracy: 0.63612s - loss: 0.6418 - sparse_ca
Epoch 3/10
719/719 [==============================] - 77s 108ms/step - loss: 0.5499 - sparse_categorical_accuracy: 0.7219
Epoch 4/10
719/719 [==============================] - 77s 108ms/step - loss: 0.3975 - sparse_categorical_accuracy: 0.82142s - loss: 0.3966 - sparse_c
Epoch 5/10
719/719 [==============================] - 77s 108ms/step - loss: 0